# Correlation between life expectancy and fertility globally 1960-2015

## Outline
- load fertility, life expectancy and population from gapminder 
- slice columns 1960-2015
- merge them together via stack and unstack 
- create individual scatter plots and histograms for each year 
- concatenate them together into respective GIFs

### Loading data and cleaning up data

In [2]:
#importing data frames 
import pandas as pd
fert=pd.read_csv("gapminder_total_fertility.csv", index_col=0)
life=pd.read_excel("gapminder_lifeexpectancy.xlsx", index_col=0)
pop=pd.read_excel('gapminder_population.xlsx', index_col=0)

In [8]:
#check each df dimensions 
print('life df: ' + str(life.shape))
print('fert df: ' + str(fert.shape))
print('pop df: ' + str(pop.shape))

life df: (260, 217)
fert df: (260, 216)
pop df: (275, 81)


In [11]:
#verify that column names are integers and strings 
print('life columns: ' + str(life.columns))
print('fert columns: ' + str(fert.columns))
print('pop columns: ' + str(pop.columns))

life columns: Int64Index([1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809,
            ...
            2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016],
           dtype='int64', length=217)
fert columns: Index(['1800', '1801', '1802', '1803', '1804', '1805', '1806', '1807', '1808',
       '1809',
       ...
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015'],
      dtype='object', length=216)
pop columns: Int64Index([1800, 1810, 1820, 1830, 1840, 1850, 1860, 1870, 1880, 1890, 1900,
            1910, 1920, 1930, 1940, 1950, 1951, 1952, 1953, 1954, 1955, 1956,
            1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967,
            1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
            1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
            1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
            2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008

In [24]:
#change fert column names from string to integer
colNames=[int(x) for x in fert.columns]
fert.set_axis(axis=1, labels=colNames, inplace=True)
#verify fert column names are now integers
print('fert columns: ' + str(fert.columns))

fert columns: Int64Index([1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809,
            ...
            2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015],
           dtype='int64', length=216)


In [23]:
#slicing 1960-2015 columns in all 3 dfs
targetCols=list(map(int, range(1960, 2016)))
fert2=fert[targetCols]
life2=life[targetCols]
pop2=pop[targetCols]#verify new dfs dimensions 
print('life df: ' + str(life2.shape))
print('fert df: ' + str(fert2.shape))
print('pop df: ' + str(pop2.shape))

life df: (260, 56)
fert df: (260, 56)
pop df: (275, 56)


### Merging fertility, population and life expectancy together via stack and unstack 

In [22]:
#folding the year columns towards the country index for each df
sfert2=fert2.stack()
slife2=life2.stack()
spop2=pop2.stack()

In [38]:
#create dictionary for merging the 3 dfs into 1 large one 
d={'fertility':sfert2, 'lifeExp':slife2, 'population':spop2} 
#create a large df from the 3 smaller ones where country and year are indices 
df1=pd.DataFrame(data=d)
#fold the df again into a series. country, year and category(fertility, lifeExp, population) are indices
df2=df1.stack()

### Creating individual plots for 1960-2015
- quick note: normalized population data so can properly see how country population affects life expectancy and fertility correlation

#### Normalization Formula(* 5000):
$$\frac{x-x_{min}}{x_{max}-x_{min}}$$

In [77]:
#use for loop to build and store each scatter plot 
import pylab as plt
for i in targetCols:
    ##partially unfold df by pulling years out of indices
    df3=df2.unstack(1)
    ##slice the individual year from main df 
    df3=df3[i]
    ##unfold the df further by pulling category from indices
    df3=df3.unstack(1)
    ##normalize population column 
    df3['popNorm']=((df3['population']-df3['population'].min())/
                    (df3['population'].max()-df3['population'].min()))*5000
    ##create a color map for plot
    cmap=plt.get_cmap('tab20', lut=len(df3)).colors
    ##building plot and adding 
    df3.plot.scatter('fertility', 'lifeExp', s=df3['popNorm'], alpha=0.5, c=cmap, figsize=(12,8))
    plt.title('Life Expectancy vs Fertility in ' + str(i))
    plt.ylabel('life expectancy')
    plt.axis([0,10,0,90])
    plt.savefig('scatter'+str(i)+'.png')
    plt.close()

In [78]:
#concatenate all scatter png images into a gif
import imageio
images=[]
for i in targetCols:
    filename='scatter{}.png'.format(i)
    images.append(imageio.imread(filename))
imageio.mimsave('scatter.gif', images, fps=20) 

In [75]:
#use similiar for loop to create life expectancy and fertility histograms 
for i in targetCols:
    ##prepare df for graphs
    df3=df2.unstack(1)
    df3=df3[i]
    df3=df3.unstack(1)
    ##build "canvas" and ID the 2 graphs
    fig, (g1, g2)=plt.subplots(nrows=1, ncols=2, figsize=(12,8))
    fig.suptitle('Life Expectancy and Frequency at ' + str(i))
    ##build histogram 1
    g1.hist(df3['fertility'], facecolor='red', edgecolor='black', linewidth=1.5)
    g1.set_xlim([0,10])
    g1.set_ylim([0,80])
    g1.set_ylabel('frequency')
    g1.set_xlabel('fertility rate')
    ##build histogram 2
    g2.hist(df3['lifeExp'], facecolor='blue', edgecolor='black', linewidth=1.5)
    g2.set_xlim([0,90])
    g2.set_ylim([0,60])
    g2.set_xlabel('Max life span')
    plt.savefig('hist'+str(i)+'.png')
    plt.close()

In [76]:
#concatenate all hist png images into a gif
import imageio
images=[]
for i in targetCols:
    filename='hist{}.png'.format(i)
    images.append(imageio.imread(filename))
imageio.mimsave('histogram.gif', images, fps=20) 